In [1]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM


2022-09-16 15:30:34.648429: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-09-16 15:30:34.653008: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/anaconda3-2021-05/lib:/cm/shared/apps/pbspro-ce/19.1.3/lib/
2022-09-16 15:30:34.653028: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## inspiration [here](https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/)

In [2]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [3]:
def ld_data(data,n_days,frac):

    dataset=data.values

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled = scaler.fit_transform(dataset)

    x_scaled = scaled[:,:data.shape[1]-1].astype('float32') #.drop('diag1',axis=1).values.astype('float32')
    y_scaled = scaled[:,data.shape[1]-1:].astype('float32') #['diag1'].values.astype('float32')
    # integer encode direction
    encoder = LabelEncoder()
    # values[:,4] = encoder.fit_transform(values[:,4])
    # ensure all data is float
    # values = values.astype('float32')
    # normalize features


    # y_scaled = scaler.fit_transform(y_values)

    # specify the number of lag days
    n_days = 7
    n_features =  x_scaled.shape[1]
    # frame as supervised learning

    x_reframed = series_to_supervised(x_scaled, n_days, 1).values
    y_reframed = y_scaled #series_to_supervised(y_scaled, n_days, 1).values
    y_reframed = y_reframed[:x_reframed.shape[0]]
    # print(reframed.shape)

    # drop=np.arange(n_features+1,(2*n_features),1)
    # reframed.drop(reframed.columns[drop], axis=1, inplace=True)


    # split into train and test sets
    # values = reframed.values

    n_train_days = 365 * round((len(x_scaled)/365)*.5) #* 24
    n_obs = n_days * n_features
    train_X = x_reframed[:n_train_days, :]
    test_X = x_reframed[n_train_days:, :]

    train_y = y_reframed[:n_train_days]
    test_y = y_reframed[n_train_days:]

    # split into input and outputs

    train_X = train_X[:, :n_obs]#, train_y[:n_obs]
    test_X = test_X[:, :n_obs]#, test_y[:n_obs]

    # print(train_X.shape, len(train_X), train_y.shape)
    # reshape input to be 3D [samples, timesteps, features]
    train_X = train_X.reshape((train_X.shape[0], n_days, n_features))
    test_X = test_X.reshape((test_X.shape[0], n_days, n_features))
    print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
    
    
    return train_X, train_y,test_X,test_y,n_features,scaler

In [4]:
def run_mdl(train_X, train_y,test_X,test_y,YY,var,site,n_days,n_features,scaler):
    # design network
    model = Sequential()
    model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dense(1))
    model.compile(loss='mae', optimizer='adam')
    # fit network
    if YY=='shuf':
        train_y=np.random.permutation(train_y)
    elif YY=='orig':
        train_y=train_y

    history = model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=0, shuffle=True)

    # plot history
    pyplot.plot(history.history['loss'], label='train')
    pyplot.plot(history.history['val_loss'], label='test')
    pyplot.legend()
    pyplot.title(var+'_'+YY+'_loss_'+site)
    # pyplot.show()
    plt.savefig('NNtest/'+var+'_'+YY+'_loss_'+site+'.png')
    plt.clf()
    
    # make a prediction
    yhat = model.predict(test_X)
    
    test_X2 = test_X.reshape((test_X.shape[0], n_days*n_features))
    # # invert scaling for forecast
    inv_yhat = concatenate((yhat, test_X2[:, -(n_features):]), axis=1)
    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:,0]
    # invert scaling for actual


    test_y2 = test_y.reshape(test_y.shape[0]*test_y.shape[1])
    inv_y = concatenate((test_y, test_X2[:, -(n_features):]), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    inv_y = inv_y[:,0]

    print([inv_yhat.shape,test_y2.shape,inv_y.shape])

    # calculate MSE
    mse = (mean_squared_error(inv_y, inv_yhat))
    print('Test MSE: %.3f' % mse)

    plt.plot(np.rint(inv_y), alpha = .5,label='pred')
    plt.plot(np.double(inv_yhat), alpha = .5,label='valid')
    plt.title(site+'_'+var+'_'+YY+ 'MSE: '+str(mse))
    plt.legend()
    plt.savefig('NNtest/'+site+'_'+var+'_'+YY+'_mse.png')
    plt.clf()
    return mse
    

In [5]:
var_list = []
mse_list = []
site_list = []
order_list = []

for i in ['all','ae','poll']:
    for SS in ['orig','shuf']:
        data=pd.read_parquet('~/run/AXA_AE_app/AE_AXA_poll-ae.parquet')#, compression='GZIP')#.mode('overwrite')
        data=data[data['cd9_loc']=='Queen Elizabeth Hospital']
        site='QEH'
        data=data.groupby(['cd9_loc','date']).agg('mean')[['pm25','pm10','o3','no2','so2','co','age','sex','diag1']]
        # data=data.groupby(['date']).agg('mean')[['pm25','pm10','o3','no2','so2','co','age','sex','diag1']]

        # data=data.groupby(['date']).agg({'pm25':'mean','pm10':'mean','o3':'mean','no2':'mean','so2':'mean','co':'mean','age':'mean','sex':'mean','diag1':'mean'})
        data.age=np.round(data.age)
        data.sex=np.round(data.sex)
        # first_column = data.pop('diag1') ## comment for avg all loc
        # data.insert(0, 'diag1', first_column) ## comment for avg all loc

        # values = dataset.values
        first_column = data.pop('diag1')
        data.insert(0, 'diag1', first_column)
        if i=='all':
            data=data
        elif i=='ae':
            data=data[['diag1','age','sex']]
        elif i =='poll':
            data[['diag1','pm25','pm10','o3','no2','so2','co']]#drop(['age','sex'],axis=1,inplace=True)
        n_days=7
        train_X, train_y,test_X,test_y,n_features,scaler=ld_data(data,n_days,.7)
        mse=run_mdl(train_X, train_y,test_X,test_y,SS,i,site,n_days,n_features,scaler)
        
        var_list.append(i)
        mse_list.append(mse)
        site_list.append(site)
        order_list.append(SS)
        print(site+'_'+i+'_'+SS+ 'MSE: '+str(mse))
        
df = pd.DataFrame(list(zip(var_list , mse_list,site_list,order_list)), 
           columns =['var', ',mse','site','order'])

df.to_csv('NNtest/NN_mse.txt',sep='\t')

(1095, 7, 8) (1095, 1) (786, 7, 8) (786, 1)


2022-09-16 15:30:36.608054: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-09-16 15:30:36.608096: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (hpc-hn001): /proc/driver/nvidia/version does not exist
2022-09-16 15:30:36.608424: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


25/25 [==============================] - 0s 1ms/step
[(786,), (786,), (786,)]
Test MSE: 218.641
QEH_all_origMSE: 218.64139
(1095, 7, 8) (1095, 1) (786, 7, 8) (786, 1)
25/25 [==============================] - 0s 2ms/step
[(786,), (786,), (786,)]
Test MSE: 219.786
QEH_all_shufMSE: 219.7857
(1095, 7, 2) (1095, 1) (837, 7, 2) (837, 1)
27/27 [==============================] - 0s 2ms/step
[(837,), (837,), (837,)]
Test MSE: 215.662
QEH_ae_origMSE: 215.66241
(1095, 7, 2) (1095, 1) (837, 7, 2) (837, 1)
27/27 [==============================] - 0s 2ms/step
[(837,), (837,), (837,)]
Test MSE: 216.603
QEH_ae_shufMSE: 216.60258
(1095, 7, 8) (1095, 1) (786, 7, 8) (786, 1)
25/25 [==============================] - 0s 1ms/step
[(786,), (786,), (786,)]
Test MSE: 219.655
QEH_poll_origMSE: 219.65489
(1095, 7, 8) (1095, 1) (786, 7, 8) (786, 1)
25/25 [==============================] - 0s 2ms/step
[(786,), (786,), (786,)]
Test MSE: 218.729
QEH_poll_shufMSE: 218.72879


<Figure size 432x288 with 0 Axes>